In [1]:
!pip install pypdf transformers sentence-transformers openai langchain_community langchain
!pip insatll pyyaml

ERROR: unknown command "insatll" - maybe you meant "install"


In [11]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

/Users/bhagyeshshinde/Documents/new_test_project/env/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:

from google.colab import files

# Upload a file
uploaded = files.upload()

# List the uploaded files
for filename in uploaded.keys():
    print(f'Uploaded file: {filename}')

Saving Data Scientist Job Description.pdf to Data Scientist Job Description.pdf
Uploaded file: Data Scientist Job Description.pdf


In [12]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Data Scientist Job Description.pdf")
pages = loader.load()

In [13]:
def return_content(pages):
  docs = []
  for page in pages:
    content_page=page.page_content
    docs.append(content_page)
  return docs

content = return_content(pages)

In [7]:
content

["Data Scientist Job Description\nThis job description is comprehensive and very well structured! It clearly outlines the responsibilities,\nrequired qualiﬁcations, and company values. It eﬀectively conveys the role and the opportunities for\ngrowth at Krowstel. The only thing that might be worth considering is to condense some sections\nslightly, especially if you're sharing this on platforms with word limits or where readers may skim.\nOtherwise, it looks great as is!\nLet me know if you'd like help with any further edits or adjustm ents.\nHere’s a more concise and engaging rewrite of the job description:\nJob Title: Data Scientist\nCompany Overview\nAt Krowstel, we are passionate about leveraging data to create innovative solutions that drive\nbusiness value. We’re a forward-thinking company that fosters collaboration, continuous learning, and\nexcellence. Our team tackles complex challenges, delivering actionable insights to shape our data-\ndriven future. We're looking for a skill

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  =100,
    length_function = len,
)
docs = [Document(page_content=text) for text in content]
docs = text_splitter.split_documents(docs)


In [15]:
docs[0].page_content

"Data Scientist Job Description\nThis job description is comprehensive and very well structured! It clearly outlines the responsibilities,\nrequired qualiﬁcations, and company values. It eﬀectively conveys the role and the opportunities for\ngrowth at Krowstel. The only thing that might be worth considering is to condense some sections\nslightly, especially if you're sharing this on platforms with word limits or where readers may skim.\nOtherwise, it looks great as is!"

In [16]:
def doc_chunked_data(data):
  chunks = []
  for chunk in data:
    chunks.append(chunk.page_content) # Append each chunk to the list
  return chunks


In [17]:
chunked_data = doc_chunked_data(docs)
#

In [18]:
embeddings = model.encode(chunked_data)

In [14]:
embeddings

array([[-0.03380662,  0.04441603,  0.04133575, ...,  0.00822239,
        -0.04383241,  0.00327837],
       [-0.06058735,  0.08011168,  0.05016645, ..., -0.04105315,
        -0.05207069,  0.0504959 ],
       [-0.02314058, -0.04701788,  0.03898378, ..., -0.05880738,
        -0.04845572,  0.02383633],
       ...,
       [ 0.01638365, -0.04651208, -0.04778657, ..., -0.0070955 ,
        -0.00301832,  0.05460208],
       [-0.06190976,  0.00301347,  0.03616453, ..., -0.01206007,
        -0.02717156,  0.09032957],
       [-0.02418642,  0.00960029, -0.06778625, ...,  0.04549592,
         0.06774611,  0.06348297]], dtype=float32)

In [19]:
questions ='''
```yaml
Job_Description:
  SALARY:
    Q1: What salary and benefits does Krowstel offer for the Data Scientist position?
    A1:
    Q2: Does Krowstel provide flexible working arrangements, and what are they?
    A2:
  EXPERIENCE:
    Q1: How many years of experience are required for the Data Scientist role at Krowstel?
    A1:
    Q2: "What kind of experience is preferred for candidates applying to this Data Scientist role (e.g., NLP, time-series data)?"
    A2:
  EDUCATION:
    Q1: "What is the minimum educational requirement for the Data Scientist position at Krowstel?"
    A1
    Q2: "Is a PhD considered a plus for this role?"
    A2:
  RESPONSIBILITIES:
    Q1: "What are the key responsibilities of a Data Scientist at Krowstel regarding data collection and preprocessing?"
    A1:
    Q2: "How does the role involve collaboration with other teams, such as engineers and product teams?"
    A2:
  SKILLS:
    Q1: "Which programming languages and data libraries are essential for this position?"
    A1:
    Q2: "What cloud platforms should the candidate be familiar with for the Data Scientist role?"
    A2:
```
'''

In [20]:
BASE_URL= "https://api.groq.com/openai/v1"
API_KEY = "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"

In [21]:
system_prompt = '''
  [INTRODUCTION]: You are expert in question answering , when you get the questions and get the relevant context or text corpus , you simply find out the relevant answer and create a yaml file with the
  question and the answer and print on the screen.

  Do not print any verbose and system generated text only simply print the Question and answer along with the subtopics provided during the questions in the yaml file.
  Do not print this verbose "Here is the answer in YAML format:"
  Only valid YAML print not other single text
'''

In [22]:
user_prompt = f'''

[CONTEXT]:{chunked_data}

[QUESTIONS]:{questions}
You should return answer in yaml format.
'''

In [23]:
from openai import OpenAI

client = OpenAI(
    api_key = API_KEY,
  base_url = BASE_URL
)
model="llama-3.1-70b-versatile"
def get_completion(system_p, user_p):
  response = client.chat.completions.create(
   model = model,
    messages=[
      {"role": "system", "content": system_p},
      {"role": "user", "content": user_p}
    ],
    temperature=0.5,
    stream=False,
  )
  return response.choices[0].message.content
  

In [24]:
llm_responce=get_completion(system_prompt, user_prompt)

In [25]:
print(llm_responce)

```yml
Job_Description:
  SALARY:
    Q1: What salary and benefits does Krowstel offer for the Data Scientist position?
    A1: Competitive salary and benefits
    Q2: Does Krowstel provide flexible working arrangements, and what are they?
    A2: Flexible working arrangements, including remote options
  EXPERIENCE:
    Q1: How many years of experience are required for the Data Scientist role at Krowstel?
    A1: 3+ years
    Q2: "What kind of experience is preferred for candidates applying to this Data Scientist role (e.g., NLP, time-series data)?"
    A2: Experience with time-series data, NLP techniques, and A/B testing
  EDUCATION:
    Q1: "What is the minimum educational requirement for the Data Scientist position at Krowstel?"
    A1: Bachelor’s or Master’s in Data Science, Statistics, Computer Science, Mathematics, or a related field
    Q2: "Is a PhD considered a plus for this role?"
    A2: Yes
  RESPONSIBILITIES:
    Q1: "What are the key responsibilities of a Data Scientist a

In [26]:
import re
def parse_yaml(llm_response: str):
    yaml = re.findall(r"```yaml\n(.*)```", llm_response, re.DOTALL)
    if yaml:
        sql = yaml[-1]
            # self.log(title="Extracted SQL", message=f"{sql}")
        return sql
    
    yaml = re.findall(r"```yml\n(.*)```", llm_response, re.DOTALL)
    if yaml:
        sql = yaml[-1]
            # self.log(title="Extracted SQL", message=f"{sql}")
        return sql
    
    yaml = re.findall(r"```(.*)```", llm_response, re.DOTALL)
    if yaml:
        sql = yaml[-1]
            # self.log(title="Extracted SQL", message=f"{sql}")
        return sql
    
    return llm_response

In [27]:
parsed_output=parse_yaml(llm_responce)

In [28]:
import yaml
yaml_data = yaml.safe_load(parsed_output)

# Convert the YAML object back to a YAML formatted string (optional)
yaml_formatted = yaml.dump(yaml_data, default_flow_style=False, sort_keys=False)

# Print the YAML formatted string
print(yaml_formatted)

# Optionally, write the YAML output to a file
with open('job_description.yaml', 'w') as file:
    yaml.dump(yaml_data, file, default_flow_style=False, sort_keys=False)

Job_Description:
  SALARY:
    Q1: What salary and benefits does Krowstel offer for the Data Scientist position?
    A1: Competitive salary and benefits
    Q2: Does Krowstel provide flexible working arrangements, and what are they?
    A2: Flexible working arrangements, including remote options
  EXPERIENCE:
    Q1: How many years of experience are required for the Data Scientist role at Krowstel?
    A1: 3+ years
    Q2: What kind of experience is preferred for candidates applying to this Data
      Scientist role (e.g., NLP, time-series data)?
    A2: Experience with time-series data, NLP techniques, and A/B testing
  EDUCATION:
    Q1: What is the minimum educational requirement for the Data Scientist position
      at Krowstel?
    A1: "Bachelor\u2019s or Master\u2019s in Data Science, Statistics, Computer Science,\
      \ Mathematics, or a related field"
    Q2: Is a PhD considered a plus for this role?
    A2: true
  RESPONSIBILITIES:
    Q1: What are the key responsibilities o

In [37]:
import json
yaml_data = yaml.safe_load(yaml_formatted)

# Initialize a list to store key-value pairs
data = []

# Loop through the dictionary and store formatted key-value pairs in `data`
for key, value in yaml_data.items():
    print(f"{key} : {value}")
    data.append({key: value})




Job_Description : {'SALARY': {'Q1': 'What salary and benefits does Krowstel offer for the Data Scientist position?', 'A1': 'Competitive salary and benefits', 'Q2': 'Does Krowstel provide flexible working arrangements, and what are they?', 'A2': 'Flexible working arrangements, including remote options'}, 'EXPERIENCE': {'Q1': 'How many years of experience are required for the Data Scientist role at Krowstel?', 'A1': '3+ years', 'Q2': 'What kind of experience is preferred for candidates applying to this Data Scientist role (e.g., NLP, time-series data)?', 'A2': 'Experience with time-series data, NLP techniques, and A/B testing'}, 'EDUCATION': {'Q1': 'What is the minimum educational requirement for the Data Scientist position at Krowstel?', 'A1': 'Bachelor’s or Master’s in Data Science, Statistics, Computer Science, Mathematics, or a related field', 'Q2': 'Is a PhD considered a plus for this role?', 'A2': True}, 'RESPONSIBILITIES': {'Q1': 'What are the key responsibilities of a Data Scient

In [38]:
data

[{'Job_Description': {'SALARY': {'Q1': 'What salary and benefits does Krowstel offer for the Data Scientist position?',
    'A1': 'Competitive salary and benefits',
    'Q2': 'Does Krowstel provide flexible working arrangements, and what are they?',
    'A2': 'Flexible working arrangements, including remote options'},
   'EXPERIENCE': {'Q1': 'How many years of experience are required for the Data Scientist role at Krowstel?',
    'A1': '3+ years',
    'Q2': 'What kind of experience is preferred for candidates applying to this Data Scientist role (e.g., NLP, time-series data)?',
    'A2': 'Experience with time-series data, NLP techniques, and A/B testing'},
   'EDUCATION': {'Q1': 'What is the minimum educational requirement for the Data Scientist position at Krowstel?',
    'A1': 'Bachelor’s or Master’s in Data Science, Statistics, Computer Science, Mathematics, or a related field',
    'Q2': 'Is a PhD considered a plus for this role?',
    'A2': True},
   'RESPONSIBILITIES': {'Q1': 'Wh

In [39]:
import json
# Function to gather key-value pairs for a section
def gather_key_value_pairs(job_description, section):
    section_data = {}
    if section in job_description:
        for key, value in job_description[section].items():
            section_data[key] = value
    return section_data

# Access the Job_Description data
job_description = data[0]["Job_Description"]

# List of sections to extract
sections_to_extract = ["SALARY", "EXPERIENCE", "EDUCATION", "RESPONSIBILITIES", "SKILLS"]

# Dictionary to hold the extracted data
extracted_data = {}

# Collect key-value pairs for each section and store them in extracted_data
for section in sections_to_extract:
    extracted_data[section] = gather_key_value_pairs(job_description, section)

# Write the extracted data to a JSON file
output_file = "extracted_job_data.json"
with open(output_file, 'w') as json_file:
    json.dump(extracted_data, json_file, indent=4)

print(f"Data successfully written to {output_file}")

Data successfully written to extracted_job_data.json
